## 1.DB에서 데이터 로드

In [2]:
import pymysql
import pandas as pd

In [10]:
con = pymysql.connect(
    host='localhost',
    user='root',
    password='1111',
    db='beaulol',
    charset='utf8'
)

customer_type_df = pd.read_sql("SELECT userID, user_skintype,user_trouble FROM customer_type", con)
customer_regis_df = pd.read_sql("SELECT userID, prd_brand, prd_name FROM customer_regis", con)
product_info_df = pd.read_sql("SELECT prd_name, prd_brand, prd_ingredients, prd_category FROM product_info", con)
solvent_df = pd.read_sql("SELECT ingredient_kor FROM solvent", con)

con.close()

C:\Users\dldkr\AppData\Local\Temp\ipykernel_23820\896438246.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_type_df = pd.read_sql("SELECT userID, user_skintype,user_trouble FROM customer_type", con)
C:\Users\dldkr\AppData\Local\Temp\ipykernel_23820\896438246.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_regis_df = pd.read_sql("SELECT userID, prd_brand, prd_name FROM customer_regis", con)
C:\Users\dldkr\AppData\Local\Temp\ipykernel_23820\896438246.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy

In [11]:
# 회원별 피부타입
customer_type_df.head()

,userID,user_skintype,user_trouble
0,F0002,건성,민감성/아토피
1,F0003,복합성,주름/탄력
2,F0004,중성,잡티/미백/홍조/다크서클
3,F0005,건성,각질
4,F0006,중성,트러블


In [12]:
# 회원별 화장품 등록정보
customer_regis_df.head()

,userID,prd_brand,prd_name
0,F0002,얼트루,아임쏘리포마이스킨 릴렉싱 앰플
1,F0002,닥터지,에이클리어 스팟 포 페이스 세럼
2,F0002,라운드랩,1025 독도 토너
3,F0002,아벤느,트릭세라 로션
4,F0002,에스트라,아토베리어365 하이드로 수딩크림


In [13]:
# product_info_df.head()

## 2.데이터 클리닝

    [콤마 포함한 성분 이름 변경]
    1) 모든 '화장품 원료'(from 화장품원료.csv) 중에 콤마가 들어가 성분 추출  (ex)1,2헥산다이올
    2) product_info_df안에 콤마포함성분 확인 -> 4가지 성분 확인
    3) 콤마포함성분을 슬래시로 변경

    [구분자 변경 & 괄호 제거]
    우선 (콤마 스페이스, 콤마, 스페이스)로 나눠진 데이터가 가장 많다. 
    하지만 스페이스와 콤마는 한 성분내에서 구분자 말고도, 데이터에 포함되어있어 바꿀경우 문제가 생긴다.
    ex) 레티놀 (500IU/g), 쏼라쏼라(100,000ppm)

    1) 이 세 구분자를 제외한 구분자(구분자로만 쓰인 문자일 것 같은 것) 모두 콤마 스페이스(, )로 변경
    2) 성분 뒤 괄호안 내용 다 제거 (정규표현식 이용)
    3) 콤마스페이스로 구분되지않은 nd_product_info_df 생성 >> 81행 발견
    이유: 스페이스나 콤마로 구분자가 많아서 그렇다.
    4) 콤마나 스페이스 구분자를, 콤마 스페이스 구분자로 변경 
    5) 그래도 이상한 이상치들 확인. 15개 성분데이터는 '-'대시로 변경하여 머신러닝 영향을 최소화
    6) nd_product_info_df 를 원본 데이터프레임과 합침.

    product_info_df의 전처리 끝


----
    [용매와 방부제 역할을 하는 성분 제거 후, 새로운 열 'main_prd_ingredients' 생성]
    1) solvent df에서 성분안에 콤마를 가진 성분 슬래쉬로 변경후, 리스트로 정리
    2) solvent 성분을 'prd_ingredients'에서 제거후, 'main_prd_ingredients'에 저장

    [유저별 화장품 목록, 화장품 성분 목록 생성]
    - 유저별로 화장품 목록과 화장품 성분목록을 묶어서 만든 뒤, 최종 셋팅 완료된 데이터를 csv 파일로 저장.

In [14]:
'''
구분자 종류:
1) 스페이스바( )
2) 콤마 스페이스(, )
3) 콤마(,)
3) 온점 스페이스(. )
4) 골뱅이(@)
5) 별 콤마(*.)
6) 별 골뱅이(*@)
7) 별 (*)
'''

'\n구분자 종류:\n1) 스페이스바( )\n2) 콤마 스페이스(, )\n3) 콤마(,)\n3) 온점 스페이스(. )\n4) 골뱅이(@)\n5) 별 콤마(*.)\n6) 별 골뱅이(*@)\n7) 별 (*)\n'

- 1) 콤마 포함 성분 이름 변경

In [15]:
# 모든 성분들 이름 불러오기
all_ingred = pd.read_csv('./data/02.데이터수집_01크롤링_화장품국문영문_화장품원료.csv')
all_ingred.head()

,INGR_KOR_NAME,INGR_ENG_NAME,CAS_NO,ORIGIN_MAJOR_KOR_NAME,INGR_SYNONYM
0,가공소금,NaN,NaN,NaN,NaN
1,가지열매추출물,Solanum Melongena (Eggplant) Fruit Extract,84012-19-1,이 원료는 가지(Eggplant) Solanum melongena의 열매에서 추출한...,가지추출물
2,구멍쇠미역추출물,Agarum Cribrosum Extract,NaN,이 원료는 조류의 일종인 구멍쇠미역 Agarum cribosum에서 추출한 것이다.,NaN
3,루핀아미노산,Lupine Amino Acids,NaN,이 원료는 루핀 단백질의 완전 가수분해로 얻은 아미노산의 혼합물이다.,NaN
4,류신,Leucine,"328-39-2(DL-)\r,61-90-5(L-)",이 원료는 다음의 구조를 갖는 아미노산이다.,NaN


In [16]:
# 콤마를 가진 성분들 저장하기
comma_inged = all_ingred[all_ingred['INGR_KOR_NAME'].str.contains(',')==True]['INGR_KOR_NAME']
comma_inged.head()

629                5-브로모-5-나이트로-1,3-디옥산
630              2-브로모-2-나이트로프로판-1,3-디올
2035                  에틸2,2-다이메틸하이드로신남알
2524    4,4'-아이소프로필리덴다이페놀/에피클로로하이드린코폴리머
3666                         1,10-데칸다이올
Name: INGR_KOR_NAME, dtype: object

In [17]:
# 콤마 포함된 성분들
for x in comma_inged:
    matching_rows = product_info_df[product_info_df['prd_ingredients'].str.contains(x)]
    if not matching_rows.empty:
        print(x)
        display(matching_rows)
# 결과: 4개의 성분이 성분들 나누기를 방해하고 있다.
# 1,2-헥산다이올, 2-올레아미도-1,3-옥타데칸다이올, 2,3-부탄다이올, 2,4-다이메틸-3-사이클로헥신카복스알데하이드

1,2-헥산다이올


,prd_name,prd_brand,prd_ingredients,prd_category
1,1025 독도 로션,라운드랩,"정제수, 글리세린, 퀸즈랜드넛오일, 1,2-헥산다이올, 펜틸렌글라이콜, 호호바에스터...",로션
2,1025 독도 토너,라운드랩,"정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, 아이리쉬모스추출물...",스킨토너
3,"11년간, 1등 잡티세럼",아이소이,"정제수, 다마스크장미꽃수, 글리세린, 에탄올,알부틴, 베타인, 트레할로오스, 다마스...",세럼
4,1번 진정 맑게담은 청초토너,넘버즈인,"병풀추출물, 약모밀추출물, 감초뿌리추출물, 1,2-헥산다이올, 다이프로필렌글라이콜,...",스킨토너
5,3번 결광가득 에센스 토너,넘버즈인,"정제수, 다이프로필렌글라이콜, 글리세레스-26, 1,2-헥산다이올, 나이아신아마이드...",스킨토너
...,...,...,...,...
432,히알루로닉 에센스,로벡틴,"정제수, 글리세린, 메틸글루세스-20, 펜틸렌글라이콜, 부틸렌글라이콜, 나이아신아마...",세럼
433,히알루론산 레이어 물톡스 앰플,메디필,"정제수, 프로판다이올, 글리세린, 다이프로필렌글라이콜, 메틸프로판다이올, 1,2-헥...",세럼
434,히알루론산 레이어 물톡스 앰플 더블 기획,메디필,"정제수, 프로판다이올, 글리세린, 다이프로필렌글라이콜, 메틸프로판다이올, 1,2-헥...",세럼
435,힐러 턴오버 액티브 크림,리쥬란,"정제수, 글리세린, 부틸렌글라이콜, 메틸하이드로제네이티드로지네이트, 다이프로필렌글라...",크림


2-올레아미도-1,3-옥타데칸다이올


,prd_name,prd_brand,prd_ingredients,prd_category
183,비오템옴므 아쿠아파워 어드밴스드 젤 로션 100ML,비오템 옴므,정제수 다이메티콘 변성알코올 글리세린 부틸렌글라이콜 시어버터 암모늄폴리아크릴로일다이...,로션


2,3-부탄다이올


...

,prd_name,prd_brand,prd_ingredients,prd_category
39,노스카나인 트러블 클리어 토너,파티온,"정제수, 2,3-부탄다이올, 에톡시다이글라이콜, 글루코노락톤, 1,2-헥산다이올, ...",스킨토너
43,다이브인 저분자 히알루론산 수딩 크림,토리든,"정제수, 부틸렌글라이콜, 글리세린, 1,2-헥산다이올, 하이드로제네이티드다이데센, ...",크림
45,다이브인 저분자 히알루론산 토너,토리든,"정제수, 부틸렌글라이콜, 다이프로필렌글라이콜, 1,2-헥산다이올, 글리세린, 베타인...",스킨토너
58,데일리뮨 앰플 세럼,피지오겔,"정제수, 다이프로필렌글라이콜, 프로판다이올, 아스코빅애씨드, 2,3-부탄다이올, 글...",세럼
71,레드 블레미쉬 클리어 모이스처 크림,닥터지,"정제수, 글리세린, 2,3-부탄다이올, 하이드로제네이티드폴리(C6-14올레핀), 병...",크림
109,리큐어 노르데나우 시카에센스,케어존,"정제수, 글리세린, 2,3-부탄다이올, 사이클로펜타실록세인, 다이메티콘, 1,2-헥...",세럼
118,마이크로액티브 브라이트닝 앰플,숨37,"정제수*, 글리세린, 부틸렌글라이콜*, 소듐아스코빌포스페이트*, 2,3-부탄다이올*...",세럼
119,마이크로액티브 카밍 앰플,숨37,"정제수*, 변성알코올, 2,3-부탄다이올*, 글리세린*, 세린*, 1,2-헥산다이올...",세럼
140,미라클 토닝 글로우 세럼,오휘,"정제수, 프로판다이올, 글리세린, 2,3-부탄다이올, 펜틸렌글라이콜, 나이아신아마이...",세럼
196,사이언수티컬즈 데일리뮨 앰플세럼,피지오겔,"정제수, 다이프로필렌글라이콜, 프로판다이올, 아스코빅애씨드, 2,3-부탄다이올, 글...",세럼


2,4-다이메틸-3-사이클로헥신카복스알데하이드


C:\Users\dldkr\AppData\Local\Temp\ipykernel_23820\1074179305.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  matching_rows = product_info_df[product_info_df['prd_ingredients'].str.contains(x)]


,prd_name,prd_brand,prd_ingredients,prd_category
29,그린토마토 포어 리프팅 앰플 플러스,성분에디터,"정제수, 부틸렌글라이콜, 글리세린, 자일리톨, 1,2-헥산다이올, 나이아신아마이드,...",세럼


C:\Users\dldkr\AppData\Local\Temp\ipykernel_23820\1074179305.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  matching_rows = product_info_df[product_info_df['prd_ingredients'].str.contains(x)]


In [18]:
# 4개 성분 이름 안 콤마를 슬래쉬(/)로 변경
product_info_df['prd_ingredients'] = product_info_df['prd_ingredients'].str.replace('1,2-헥산다이올', '1/2-헥산다이올')
product_info_df['prd_ingredients'] = product_info_df['prd_ingredients'].str.replace('2-올레아미도-1,3-옥타데칸다이올', '2-올레아미도-1/3-옥타데칸다이올')
product_info_df['prd_ingredients'] = product_info_df['prd_ingredients'].str.replace('2,3-부탄다이올', '2/3-부탄다이올')
product_info_df['prd_ingredients'] = product_info_df['prd_ingredients'].str.replace('2,4-다이메틸-3-사이클로헥신카복스알데하이드', '2/4-다이메틸-3-사이클로헥신카복스알데하이드')

* 2. 구분자 변경 & 괄호 제거

In [19]:
# 우선 구분자 수정> ', '로
import re
product_info_df['prd_ingredients'] = product_info_df['prd_ingredients'].apply(lambda x: re.sub(r'\. |@|\*\.|\*,|\*@|\*', ', ', x))

# nd_product_info_df['prd_ingredients'] = nd_product_info_df['prd_ingredients'].str.replace('. ', ', ').str.replace('@', ', ').str.replace('*.', ', ').str.replace('*,',', ').str.replace('*@', ', ')

In [20]:
# 'prd_ingredients' 열의 문자열에서 괄호와 괄호 안의 내용을 제거

product_info_df['prd_ingredients'] = product_info_df['prd_ingredients'].apply(lambda x:\
    re.sub(r'\(.*?\)', '', x))

In [21]:
product_info_df.head()

,prd_name,prd_brand,prd_ingredients,prd_category
0,10% 나이아신아마이드 부스터 앰플,폴라초이스,"정제수, 나이아신아마이드, 아세틸글루코사민, 아스코빌글루코사이드, 부틸렌글라이콜, ...",세럼
1,1025 독도 로션,라운드랩,"정제수, 글리세린, 퀸즈랜드넛오일, 1/2-헥산다이올, 펜틸렌글라이콜, 호호바에스터...",로션
2,1025 독도 토너,라운드랩,"정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, 아이리쉬모스추출물...",스킨토너
3,"11년간, 1등 잡티세럼",아이소이,"정제수, 다마스크장미꽃수, 글리세린, 에탄올,알부틴, 베타인, 트레할로오스, 다마스...",세럼
4,1번 진정 맑게담은 청초토너,넘버즈인,"병풀추출물, 약모밀추출물, 감초뿌리추출물, 1/2-헥산다이올, 다이프로필렌글라이콜,...",스킨토너


In [22]:
# 콤마 스페이스로 구분 되지 않는 것들
nd_product_info_df = product_info_df[product_info_df['prd_ingredients'].str.split(', ').apply(len) == 1]

display(product_info_df[product_info_df['prd_ingredients'].str.split(', ').apply(len) == 1])
nd_product_info_df.info()
# 81개의 행이 이상.. 스페이스와 콤마로만 구분하는 구분자 존재. 

# product_info_df[len(product_info_df['prd_ingredients'].str.split(','))==1]
# 오류나는 이유??

,prd_name,prd_brand,prd_ingredients,prd_category
20,NEW 시카플라스트 밤B5+,라로슈포제,정제수 하이드로제네이티드폴리아이소부텐 다이메티콘 글리세린 시어버터 판테놀 옥수수전분...,크림
23,갈락토미세스 발효 에센스,믹순,갈락토미세스발효여과물 100%,세럼
35,나이아신아마이드+모과추출물 세럼,마리엔메이,"모과추출물,글리세레스-26,나이아신아마이드,1/2-헥산다이올,알지닌,카보머,에틸헥실...",세럼
36,낫츠 센텔라스카 연고 일랑일랑 15g,낫츠,-,크림
44,다이브인 저분자 히알루론산 크림,토리든,화장품법에 따라 기재해야 하는 모든 성분,크림
...,...,...,...,...
413,하이드라 크림 앰플 더블 기획,나인위시스,-하이드라 크림앰플,세럼
421,하이알루론 부스팅에센스,유세린,"정제수,변성알코올,글리세린,부틸렌글라이콜,판테놀,피이지-40하이드로제네이티드캐스터오...",세럼
423,허니 세럼,아임프롬,"글리세린,정제수,부틸렌글라이콜,다이메티콘,사이클로펜타실록세인,나이아신아마이드,피이지...",세럼
428,히아루론산 토너 더블 기획,이즈앤트리,"정제수,프로판다이올,1/2-헥산다이올,소듐하이알루로네이트,트레할로오스,소듐피씨에이,...",스킨토너


<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 20 to 430
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   prd_name         81 non-null     object
 1   prd_brand        81 non-null     object
 2   prd_ingredients  81 non-null     object
 3   prd_category     81 non-null     object
dtypes: object(4)
memory usage: 3.2+ KB


In [24]:
# 스페이스나 콤마 > 콤마 스페이스로 변경
nd_product_info_df['prd_ingredients'] = nd_product_info_df['prd_ingredients'].apply(lambda x: re.sub(r' |,',', ',x))

C:\Users\dldkr\AppData\Local\Temp\ipykernel_23820\623909897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nd_product_info_df['prd_ingredients'] = nd_product_info_df['prd_ingredients'].apply(lambda x: re.sub(r' |,',', ',x))


In [25]:
nd_product_info_df

,prd_name,prd_brand,prd_ingredients,prd_category
20,NEW 시카플라스트 밤B5+,라로슈포제,"정제수, 하이드로제네이티드폴리아이소부텐, 다이메티콘, 글리세린, 시어버터, 판테놀,...",크림
23,갈락토미세스 발효 에센스,믹순,"갈락토미세스발효여과물, 100%",세럼
35,나이아신아마이드+모과추출물 세럼,마리엔메이,"모과추출물, 글리세레스-26, 나이아신아마이드, 1/2-헥산다이올, 알지닌, 카보머...",세럼
36,낫츠 센텔라스카 연고 일랑일랑 15g,낫츠,-,크림
44,다이브인 저분자 히알루론산 크림,토리든,"화장품법에, 따라, 기재해야, 하는, 모든, 성분",크림
...,...,...,...,...
413,하이드라 크림 앰플 더블 기획,나인위시스,"-하이드라, 크림앰플",세럼
421,하이알루론 부스팅에센스,유세린,"정제수, 변성알코올, 글리세린, 부틸렌글라이콜, 판테놀, 피이지-40하이드로제네이티...",세럼
423,허니 세럼,아임프롬,"글리세린, 정제수, 부틸렌글라이콜, 다이메티콘, 사이클로펜타실록세인, 나이아신아마이...",세럼
428,히아루론산 토너 더블 기획,이즈앤트리,"정제수, 프로판다이올, 1/2-헥산다이올, 소듐하이알루로네이트, 트레할로오스, 소듐...",스킨토너


In [27]:
# 아예 이상치 15개 성분 데이터는 prd_ingredients에 대시로 변경
remove_index = [36,44,146,147,236,237,240,278,312,326,327,349,401,413,430]

for i in remove_index:
    nd_product_info_df.loc[i,'prd_ingredients'] = '-'
nd_product_info_df

,prd_name,prd_brand,prd_ingredients,prd_category
20,NEW 시카플라스트 밤B5+,라로슈포제,"정제수, 하이드로제네이티드폴리아이소부텐, 다이메티콘, 글리세린, 시어버터, 판테놀,...",크림
23,갈락토미세스 발효 에센스,믹순,"갈락토미세스발효여과물, 100%",세럼
35,나이아신아마이드+모과추출물 세럼,마리엔메이,"모과추출물, 글리세레스-26, 나이아신아마이드, 1/2-헥산다이올, 알지닌, 카보머...",세럼
36,낫츠 센텔라스카 연고 일랑일랑 15g,낫츠,-,크림
44,다이브인 저분자 히알루론산 크림,토리든,-,크림
...,...,...,...,...
413,하이드라 크림 앰플 더블 기획,나인위시스,-,세럼
421,하이알루론 부스팅에센스,유세린,"정제수, 변성알코올, 글리세린, 부틸렌글라이콜, 판테놀, 피이지-40하이드로제네이티...",세럼
423,허니 세럼,아임프롬,"글리세린, 정제수, 부틸렌글라이콜, 다이메티콘, 사이클로펜타실록세인, 나이아신아마이...",세럼
428,히아루론산 토너 더블 기획,이즈앤트리,"정제수, 프로판다이올, 1/2-헥산다이올, 소듐하이알루로네이트, 트레할로오스, 소듐...",스킨토너


In [28]:
# nd_product_info_df를 사용하여 원본 데이터프레임을 업데이트
# nd_product_info_df에 있는 모든 인덱스에 해당하는 행의 'prd_ingredients' 값을 대체

for idx in nd_product_info_df.index:
    product_info_df.at[idx, 'prd_ingredients'] = nd_product_info_df.at[idx, 'prd_ingredients']

# product_info_df.iloc[183]


In [29]:
# 완벽한 데이터셋 준비 끝
product_info_df.head()

,prd_name,prd_brand,prd_ingredients,prd_category
0,10% 나이아신아마이드 부스터 앰플,폴라초이스,"정제수, 나이아신아마이드, 아세틸글루코사민, 아스코빌글루코사이드, 부틸렌글라이콜, ...",세럼
1,1025 독도 로션,라운드랩,"정제수, 글리세린, 퀸즈랜드넛오일, 1/2-헥산다이올, 펜틸렌글라이콜, 호호바에스터...",로션
2,1025 독도 토너,라운드랩,"정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, 아이리쉬모스추출물...",스킨토너
3,"11년간, 1등 잡티세럼",아이소이,"정제수, 다마스크장미꽃수, 글리세린, 에탄올,알부틴, 베타인, 트레할로오스, 다마스...",세럼
4,1번 진정 맑게담은 청초토너,넘버즈인,"병풀추출물, 약모밀추출물, 감초뿌리추출물, 1/2-헥산다이올, 다이프로필렌글라이콜,...",스킨토너


In [30]:
# 정제수 포함 하기라도 하면 다 제거해라. (■ 정제수, 혹은 정제수,로 연결된 것)
product_info_df['prd_ingredients'] = product_info_df['prd_ingredients'].apply(lambda x: ', '.join([ingredient.strip() for \
    ingredient in x.split(', ') if '정제수' not in ingredient]))


* 3. 용매와 방부제 제거 후, 핵심 성분열 저장

In [32]:
# 솔벤트도 한 성분내에 콤마있을경우, 슬래시로 변경
solvent_df['ingredient_kor'] = solvent_df['ingredient_kor'].str.replace(',','/')

# 솔벤트 리스트 정의
solvent_lst = solvent_df['ingredient_kor'].tolist()
solvent_lst

['사이클로테트라실록세인',
 '테트라데센',
 '메틸트라이메티콘',
 '옥타데케인',
 '피이지-18',
 '사이클로헥사실록세인',
 '피이지/피피지-6/2코폴리머',
 '에틸에터',
 '헥산다이올',
 '다이부틸아디페이트',
 '다이메틸아이소소바이드',
 '하이드로제네이티드폴리데센',
 '피피지-55글리세릴에터',
 '퍼플루오로퍼하이드로페난트렌',
 '아이소옥탄',
 '퍼플루오로다이메틸사이클로헥산',
 '페녹시프로판다이올',
 '다이옥솔레인',
 '사이클로펜타실록세인',
 '3-메톡시부탄올',
 '트라이부틸시트레이트',
 '상어간오일',
 '다이에틸옥살레이트',
 '헥실렌글라이콜',
 '다이에톡시에틸석시네이트',
 '에틸아세테이트',
 '트라이아세틴',
 '트라이에틸포스페이트',
 '벤질벤조에이트',
 '프로필렌글라이콜부틸에터',
 '다이메틸석시네이트',
 '엠이케이',
 '글리세레스-18벤조에이트',
 '자일렌',
 '펜타플루오로프로판',
 '메톡시아이소프로판올',
 '스쿠알렌',
 '다이아이소옥틸아디페이트',
 '피피지-3',
 'C16-17알킬벤조에이트',
 '미비케이',
 'C10-12알케인/사이클로알케인',
 '사이클로헥산온',
 '다이-피피지-3미리스틸에터아디페이트',
 '피이지/피피지-200/70코폴리머',
 'C14-19알케인',
 '톨루엔',
 '솔베스-30',
 '테트라하이드로퍼퓨릴아세테이트',
 'C10-18트라이글리세라이즈',
 '다이에틸세바케이트',
 '다이부틸옥살레이트',
 '아이소부틸벤조에이트',
 '퍼플루오로데칼린',
 '옥틸도데칸올',
 '다이메틸프탈레이트',
 '피이지-4',
 '피이지-240',
 '퍼플루오로퍼하이드로벤질테트랄린',
 '글리세레스-7락테이트',
 '메틸부텐',
 '피피지-7',
 '피이지/피피지-25/30코폴리머',
 '피이지-135',
 '에틸피롤리돈',
 '스테아릴벤조에이트',
 '다이트라이데실아디페이트',
 '데카플루오로펜탄',
 '사이클로헵타실록세인',
 '글라이콜',
 '스타이렌',
 '피이지-60

In [33]:
# 'main_prd_ingredients' 열 추가 및 초기화
product_info_df['main_prd_ingredients'] = ''

# 'prd_ingredients' 열의 문자열에서 'solvent_lst'의 요소를 제거
product_info_df['main_prd_ingredients'] = product_info_df['prd_ingredients'].apply(lambda x:\
    ', '.join([item.strip() for item in x.split(', ') if item not in solvent_lst]))

# 결과 확인
print(product_info_df['main_prd_ingredients'])

0      나이아신아마이드, 아세틸글루코사민, 아스코빌글루코사이드, 포스포리피드, 소듐하이알루...
1      퀸즈랜드넛오일, 호호바에스터, 아이리쉬모스추출물, 사탕수수추출물, 해수, 하이알루로...
2      아이리쉬모스추출물, 사탕수수추출물, 해수, 프로테아제, 베타인, 판테놀, 에틸헥실글...
3      다마스크장미꽃수, 에탄올,알부틴, 베타인, 트레할로오스, 다마스크장미꽃오일, 비타민...
4      병풀추출물, 약모밀추출물, 감초뿌리추출물, 프로판다이올,부틸렌글라이콜, 히아신스전초...
                             ...                        
432    메틸글루세스-20, 나이아신아마이드, 솔비톨, 아데노신, 카페인, 소듐하이알루로네이...
433    나이아신아마이드, 다시마추출물, 한련초잎추출물, 병풀뿌리추출물, 에난티아 클로란타껍...
434    나이아신아마이드, 다시마추출물, 한련초잎추출물, 병풀뿌리추출물, 에난티아 클로란타껍...
435    메틸하이드로제네이티드로지네이트, 다이스타치포스페이트, 다이카프릴릴카보네이트, 세테아...
436    서양산딸기잎추출물, 스페인감초뿌리추출물, 포트마리골드꽃추출물, 알로에베라잎즙, 로우...
Name: main_prd_ingredients, Length: 437, dtype: object


In [34]:
product_info_df.head()

,prd_name,prd_brand,prd_ingredients,prd_category,main_prd_ingredients
0,10% 나이아신아마이드 부스터 앰플,폴라초이스,"나이아신아마이드, 아세틸글루코사민, 아스코빌글루코사이드, 부틸렌글라이콜, 포스포리피...",세럼,"나이아신아마이드, 아세틸글루코사민, 아스코빌글루코사이드, 포스포리피드, 소듐하이알루..."
1,1025 독도 로션,라운드랩,"글리세린, 퀸즈랜드넛오일, 1/2-헥산다이올, 펜틸렌글라이콜, 호호바에스터, 아이리...",로션,"퀸즈랜드넛오일, 호호바에스터, 아이리쉬모스추출물, 사탕수수추출물, 해수, 하이알루로..."
2,1025 독도 토너,라운드랩,"부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, 아이리쉬모스추출물, 사탕수...",스킨토너,"아이리쉬모스추출물, 사탕수수추출물, 해수, 프로테아제, 베타인, 판테놀, 에틸헥실글..."
3,"11년간, 1등 잡티세럼",아이소이,"다마스크장미꽃수, 글리세린, 에탄올,알부틴, 베타인, 트레할로오스, 다마스크장미꽃오...",세럼,"다마스크장미꽃수, 에탄올,알부틴, 베타인, 트레할로오스, 다마스크장미꽃오일, 비타민..."
4,1번 진정 맑게담은 청초토너,넘버즈인,"병풀추출물, 약모밀추출물, 감초뿌리추출물, 1/2-헥산다이올, 다이프로필렌글라이콜,...",스킨토너,"병풀추출물, 약모밀추출물, 감초뿌리추출물, 프로판다이올,부틸렌글라이콜, 히아신스전초..."


In [35]:
## 수정본 불러오기

In [36]:
# 수정본 불러오기
product_info_df = pd.read_excel('./data/03.머신러닝_02클러스터링_전처리_전처리중간수정본.xlsx',index_col=0)

In [37]:
product_info_df.head()

,prd_name,prd_brand,prd_ingredients,prd_category,main_prd_ingredients
0,10% 나이아신아마이드 부스터 앰플,폴라초이스,"나이아신아마이드, 아세틸글루코사민, 아스코빌글루코사이드, 부틸렌글라이콜, 포스포리피...",세럼,"나이아신아마이드, 아세틸글루코사민, 아스코빌글루코사이드, 포스포리피드, 소듐하이알루..."
1,1025 독도 로션,라운드랩,"글리세린, 퀸즈랜드넛오일, 1/2-헥산다이올, 펜틸렌글라이콜, 호호바에스터, 아이리...",로션,"퀸즈랜드넛오일, 호호바에스터, 아이리쉬모스추출물, 사탕수수추출물, 해수, 하이알루로..."
2,1025 독도 토너,라운드랩,"부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, 아이리쉬모스추출물, 사탕수...",스킨토너,"아이리쉬모스추출물, 사탕수수추출물, 해수, 프로테아제, 베타인, 판테놀, 에틸헥실글..."
3,"11년간, 1등 잡티세럼",아이소이,"다마스크장미꽃수, 글리세린, 에탄올, 알부틴, 베타인, 트레할로오스, 다마스크장미꽃...",세럼,"다마스크장미꽃수, 에탄올, 알부틴, 베타인, 트레할로오스, 다마스크장미꽃오일, 비타..."
4,1번 진정 맑게담은 청초토너,넘버즈인,"병풀추출물, 약모밀추출물, 감초뿌리추출물, 1/2-헥산다이올, 다이프로필렌글라이콜,...",스킨토너,"병풀추출물, 약모밀추출물, 감초뿌리추출물, 프로판다이올, 부틸렌글라이콜, 히아신스전..."


## 3.머신러닝 DF 만들기

In [38]:
# 유저의 화장품 등록 정보
customer_regis_df.head()

,userID,prd_brand,prd_name
0,F0002,얼트루,아임쏘리포마이스킨 릴렉싱 앰플
1,F0002,닥터지,에이클리어 스팟 포 페이스 세럼
2,F0002,라운드랩,1025 독도 토너
3,F0002,아벤느,트릭세라 로션
4,F0002,에스트라,아토베리어365 하이드로 수딩크림


In [39]:
# 화장품 제품별 성분 정보
product_info_df.head()

,prd_name,prd_brand,prd_ingredients,prd_category,main_prd_ingredients
0,10% 나이아신아마이드 부스터 앰플,폴라초이스,"나이아신아마이드, 아세틸글루코사민, 아스코빌글루코사이드, 부틸렌글라이콜, 포스포리피...",세럼,"나이아신아마이드, 아세틸글루코사민, 아스코빌글루코사이드, 포스포리피드, 소듐하이알루..."
1,1025 독도 로션,라운드랩,"글리세린, 퀸즈랜드넛오일, 1/2-헥산다이올, 펜틸렌글라이콜, 호호바에스터, 아이리...",로션,"퀸즈랜드넛오일, 호호바에스터, 아이리쉬모스추출물, 사탕수수추출물, 해수, 하이알루로..."
2,1025 독도 토너,라운드랩,"부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, 아이리쉬모스추출물, 사탕수...",스킨토너,"아이리쉬모스추출물, 사탕수수추출물, 해수, 프로테아제, 베타인, 판테놀, 에틸헥실글..."
3,"11년간, 1등 잡티세럼",아이소이,"다마스크장미꽃수, 글리세린, 에탄올, 알부틴, 베타인, 트레할로오스, 다마스크장미꽃...",세럼,"다마스크장미꽃수, 에탄올, 알부틴, 베타인, 트레할로오스, 다마스크장미꽃오일, 비타..."
4,1번 진정 맑게담은 청초토너,넘버즈인,"병풀추출물, 약모밀추출물, 감초뿌리추출물, 1/2-헥산다이올, 다이프로필렌글라이콜,...",스킨토너,"병풀추출물, 약모밀추출물, 감초뿌리추출물, 프로판다이올, 부틸렌글라이콜, 히아신스전..."


In [40]:
# 유저 화장대df에 화장품에 맞는 성분정보 df를 합쳐주기
customer_regis_df = pd.merge(customer_regis_df, product_info_df[['prd_name','main_prd_ingredients']], \
    how='left', on='prd_name' )

In [41]:
# 화장대 DB와 화장품성분 DB 결합완료
customer_regis_df.head()

,userID,prd_brand,prd_name,main_prd_ingredients
0,F0002,얼트루,아임쏘리포마이스킨 릴렉싱 앰플,"병풀잎수, 메틸글루세스-20, 폴리글리세린-3, 병풀추출물, 무화과추출물, 다시마추..."
1,F0002,닥터지,에이클리어 스팟 포 페이스 세럼,"나이아신아마이드, 락토바실러스발효물, 암모늄폴리아크릴로일다이메틸타우레이트, 판테놀,..."
2,F0002,라운드랩,1025 독도 토너,"아이리쉬모스추출물, 사탕수수추출물, 해수, 프로테아제, 베타인, 판테놀, 에틸헥실글..."
3,F0002,아벤느,트릭세라 로션,"아벤느온천수, 글리세린, 카프릴릭/카프릭트리글리세라이드, 이소프로필팔미테이트, 프로..."
4,F0002,에스트라,아토베리어365 하이드로 수딩크림,"페닐트라이메티콘, 트레할로오스, 다이카프릴릴카보네이트, 암모늄아크릴로일다이메틸타우레..."


* 최종 머신러닝 돌릴 메인 1개의 DF 만들기

In [42]:
# 유저 피부타입 DF
customer_type_df.head()

,userID,user_skintype,user_trouble
0,F0002,건성,민감성/아토피
1,F0003,복합성,주름/탄력
2,F0004,중성,잡티/미백/홍조/다크서클
3,F0005,건성,각질
4,F0006,중성,트러블


In [43]:
# 머신러닝 돌릴 메인 DF 틀 생성
kMeans_df = customer_type_df.copy()
kMeans_df.head()

,userID,user_skintype,user_trouble
0,F0002,건성,민감성/아토피
1,F0003,복합성,주름/탄력
2,F0004,중성,잡티/미백/홍조/다크서클
3,F0005,건성,각질
4,F0006,중성,트러블


In [44]:
# 유저 화장대 DF
customer_regis_df.head()

,userID,prd_brand,prd_name,main_prd_ingredients
0,F0002,얼트루,아임쏘리포마이스킨 릴렉싱 앰플,"병풀잎수, 메틸글루세스-20, 폴리글리세린-3, 병풀추출물, 무화과추출물, 다시마추..."
1,F0002,닥터지,에이클리어 스팟 포 페이스 세럼,"나이아신아마이드, 락토바실러스발효물, 암모늄폴리아크릴로일다이메틸타우레이트, 판테놀,..."
2,F0002,라운드랩,1025 독도 토너,"아이리쉬모스추출물, 사탕수수추출물, 해수, 프로테아제, 베타인, 판테놀, 에틸헥실글..."
3,F0002,아벤느,트릭세라 로션,"아벤느온천수, 글리세린, 카프릴릭/카프릭트리글리세라이드, 이소프로필팔미테이트, 프로..."
4,F0002,에스트라,아토베리어365 하이드로 수딩크림,"페닐트라이메티콘, 트레할로오스, 다이카프릴릴카보네이트, 암모늄아크릴로일다이메틸타우레..."


In [45]:
## 유저화장대 DF: 유저별로 성분 합치기

sub_df = customer_regis_df.groupby('userID')['main_prd_ingredients'].apply(lambda x: ', '.join(x)).reset_index()
sub_df.head()

,userID,main_prd_ingredients
0,F0002,"병풀잎수, 메틸글루세스-20, 폴리글리세린-3, 병풀추출물, 무화과추출물, 다시마추..."
1,F0003,"스쿠알란, 하이드롤라이즈드효모추출물, 하이드록시에틸아크릴레이트/소듐아크릴로일다이메틸..."
2,F0004,"사과수, 글라이콜릭애씨드, 나이아신아마이드, 소듐하이드록사이드, 판테놀, 소듐하이알..."
3,F0005,"글리세레스-26, 나이아신아마이드, 아크릴레이트/C10-30알킬아크릴레이트크로스폴리..."
4,F0006,"달팽이점액여과물, 나이아신아마이드, 판테놀, 자이모모나스발효추출물, 꿀추출물, 하이..."


In [46]:
## 유저화장대 DF: 유저별로 화장품 이름 합치기

sub_df2 = customer_regis_df.groupby('userID')['prd_name'].agg(lambda x: ', '.join(x)).reset_index()
sub_df2.head()

,userID,prd_name
0,F0002,"아임쏘리포마이스킨 릴렉싱 앰플, 에이클리어 스팟 포 페이스 세럼, 1025 독도 토..."
1,F0003,"하이드라메모리 세럼, 멀티-펩타이드 + 카퍼 펩타이즈 1% 세럼, 크림스킨, 포 맨..."
2,F0004,"아하 7 화이트헤드 파워 리퀴드, 레티놀 흔적 리페어 앰플, 1번 진정 맑게담은 청..."
3,F0005,"파이브 플라워 다크스팟 클리어링 세럼, 브링그린 토닝비타20%빛세럼25mL, 화이트..."
4,F0006,"로얄 블랙 스네일 퍼스트 에센스, 모링가 타이트닝 포어 세럼, 레드수딩 시카 밸런스..."


In [47]:
## 최종 전처리끝난 KMeans_df 생성 완료

kMeans_df = pd.merge(kMeans_df,sub_df, how='left', on='userID')
kMeans_df = pd.merge(kMeans_df, sub_df2, how = 'left', on='userID')
kMeans_df.head()

,userID,user_skintype,user_trouble,main_prd_ingredients,prd_name
0,F0002,건성,민감성/아토피,"병풀잎수, 메틸글루세스-20, 폴리글리세린-3, 병풀추출물, 무화과추출물, 다시마추...","아임쏘리포마이스킨 릴렉싱 앰플, 에이클리어 스팟 포 페이스 세럼, 1025 독도 토..."
1,F0003,복합성,주름/탄력,"스쿠알란, 하이드롤라이즈드효모추출물, 하이드록시에틸아크릴레이트/소듐아크릴로일다이메틸...","하이드라메모리 세럼, 멀티-펩타이드 + 카퍼 펩타이즈 1% 세럼, 크림스킨, 포 맨..."
2,F0004,중성,잡티/미백/홍조/다크서클,"사과수, 글라이콜릭애씨드, 나이아신아마이드, 소듐하이드록사이드, 판테놀, 소듐하이알...","아하 7 화이트헤드 파워 리퀴드, 레티놀 흔적 리페어 앰플, 1번 진정 맑게담은 청..."
3,F0005,건성,각질,"글리세레스-26, 나이아신아마이드, 아크릴레이트/C10-30알킬아크릴레이트크로스폴리...","파이브 플라워 다크스팟 클리어링 세럼, 브링그린 토닝비타20%빛세럼25mL, 화이트..."
4,F0006,중성,트러블,"달팽이점액여과물, 나이아신아마이드, 판테놀, 자이모모나스발효추출물, 꿀추출물, 하이...","로얄 블랙 스네일 퍼스트 에센스, 모링가 타이트닝 포어 세럼, 레드수딩 시카 밸런스..."


In [49]:
# 최종 전처리 완료된 csv
kMeans_df.to_csv('./data/03.머신러닝_02클러스터링_전처리_전처리완료본.csv',encoding='cp949')